`PoinNet0_env` est ok ici

In [1]:
import numpy as np
import os
import tqdm

from utils import write_obj_file, read_obj_file, get_centroid, read_paths_from_txt, cross

`off_files` : Paths des fichiers .off à modifier

In [2]:
txt_file_modelnet40 = 'obj_SMPLER_files_ModelNet40.txt'
# list of OFF files paths
obj_files = read_paths_from_txt(txt_file_modelnet40); print(len(obj_files))

12311


In [3]:
def center_vertices(vertices):
    """Centers the vertices around [0, 0, 0]."""
    centroid = np.mean(vertices, axis=0)
    centered_vertices = vertices - centroid
    return centered_vertices

def center_and_scale_v2(vertices):
    """
    Centers and scales the vertices of a 3D object so that it fits within a -1 to 1 bounding box.
    Args: vertices (numpy array): Array of vertices (Nx3) where N is the number of vertices.
    Returns: numpy array: The centered and scaled vertices.
    """
    # Center the vertices by subtracting the mean
    center = np.mean(vertices, axis=0)
    centered_vertices = vertices - center

    # Find the maximum distance from the origin
    max_distance = np.max(np.linalg.norm(centered_vertices, axis=1))

    # Scale the vertices so that the object fits within the -1 to 1 range
    scaled_vertices = centered_vertices / max_distance

    return scaled_vertices


In [4]:
def center_and_scale(vertices, faces):
    """Centers and scales the vertices between -1 and 1."""
    # Calculate the centroid
    centroid = get_centroid(faces, vertices)

    # Center the vertices
    centered_vertices = vertices - centroid

    # Scale to fit between -1 and 1
    max_distance = np.max(np.linalg.norm(centered_vertices, axis=1))
    if max_distance > 0:
        scaled_vertices = centered_vertices / max_distance
    else:
        scaled_vertices = centered_vertices  # No scaling if all points are the same

    return scaled_vertices


### Transformation sur 1 fichier

In [5]:
# Example Usage
file_path = '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40/piano/train/piano_0001.obj'
output_dir = '/home/pelissier/These-ATER/Papier_international3/Dataset'

#  Read the vertices and faces from the OBJ file --> Convert from 1-based to 0-based index
vertices, faces = read_obj_file(file_path)
# Center and scale the vertices
centered_scaled_vertices = center_and_scale(vertices, faces)

# Write the centered and scaled vertices to a new OBJ file
file_name = os.path.basename(file_path)
name = os.path.splitext(file_name)[0]; extension = os.path.splitext(file_name)[1]
output_file_path = os.path.join(output_dir, name + '_centered_scaled' + extension)
#write_obj_file(output_file_path, centered_scaled_vertices, faces)

### Sur tous les fichiers

In [6]:
# Recréer la strcuture de dossiers de ModelNet40 dans ModelNet40_centered_scaled
def replicate_structure(src_dir, dest_dir):
    # Walk through the directory tree of the source directory
    for root, dirs, _ in os.walk(src_dir):
        # Construct the destination directory path
        relative_path = os.path.relpath(root, src_dir)
        dest_path = os.path.join(dest_dir, relative_path)
        
        # Create the directories in the destination
        os.makedirs(dest_path, exist_ok=True)
        #print(f"Created: {dest_path}")
        
#replicate_structure('/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40', '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_limper')

In [7]:
import concurrent.futures
import tqdm

# Function to process each file (reading, centering, scaling, writing)
def process_obj_file(file_path):
    # Read the vertices and faces from the OBJ file
    vertices, faces = read_obj_file(file_path)
    # Center and scale the vertices
    centered_scaled_vertices = center_and_scale(vertices, faces)
    # Adapt output file path
    dest_path = file_path.replace("ModelNet40", "ModelNet40_centered_scaled")
    # Get the directory and filename parts
    directory, filename = os.path.split(dest_path)
    # Modify the filename to append "_centered_scaled" before the extension
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_centered_scaled{ext}"
    # Combine the nw directory and modified filename
    output_file_path = os.path.join(directory, new_filename)
    # Write the centered and scaled vertices to a new OBJ file
    write_obj_file(output_file_path, centered_scaled_vertices, faces)
    return output_file_path

# Main function to parallelize the loop
def process_files_in_parallel(obj_files, max_workers=8):
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Use a tqdm progress bar with concurrent futures to show progress
        results = list(tqdm.tqdm(executor.map(process_obj_file, obj_files), total=len(obj_files)))
    return results


# Process the files in parallel (adjust the number of workers if necessary)
process_files_in_parallel(obj_files, max_workers=14)


100%|██████████| 12311/12311 [05:12<00:00, 39.42it/s] 


['/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0292_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0242_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0254_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0224_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0237_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0283_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/car/test/car_0218_SMPLER_centered_scaled.obj',
 '/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40

In [ ]:
obj_file